# Alternative Approaches 
To determine steps using acceleration magnitude:

In [ ]:

df['acc_magnitude'] = np.sqrt(df['ax']**2 + df['ay']**2 + df['az']**2)
df['gyro_magnitude'] = np.sqrt(df['gx']**2 + df['gy']**2 + df['gz']**2)
df['step_interval'] = df['time'].diff()

# Fill NaN values that might result from diff()
df['step_interval'] = df['step_interval'].fillna(0)

# Detect steps using acceleration magnitude
def count_peaks(series):
    peaks, _ = find_peaks(series, height=0.5)  # Adjust threshold as needed
    return len(peaks)

step_counts = df.groupby(["id", "side"])["acc_magnitude"].apply(count_peaks).unstack(fill_value=0)
step_counts.columns = ["left_steps", "right_steps"]

# Merge step counts with session features
session_features = session_features.rename(columns={"id_": "id"})
final_df = session_features.merge(step_counts, on="id")

In [ ]:
# extract some data for flutter app (since it is huge dataset and it is very mobile app static file)
df_filtered = df[(df['id'] == '033nuFnKoOjj4NeIt9FS') | 
                 (df['id'] == '0373xrf1eaJoc8IcE6Gc') | 
                 (df['id'] == '04SwmTFshylAIDUNCYTh') | 
                 (df['id'] == '0AxduuyH7QvfV841ANdD') | 
                 (df['id'] == '0bYDrU653eQr2GwcMXXw')]

# Reset the index to include it as a column
df_filtered_reset = df_filtered.reset_index()

# Convert the 'time' column to datetime and then to string with milliseconds
df_filtered_reset['time'] = pd.to_datetime(df_filtered_reset['time'], unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S.%f')

df_filtered_reset.to_json('preprocess_file_data.json', orient='records', indent=4)